In [2]:
import pandas as pd
import os
import glob
from shutil import copyfile as copy

inputPath = '../data/records_samples/'
outputPath = '../src/ext/DNorm-0.0.7/'
pklPath = '../data/interim'

if os.path.isdir(outputPath)!=1:
    os.mkdir(outputPath)

# 1.
# Load dataframe with records
Requires records.pkl file in /data/records_sample

# Load input for DNorm
Make the input data for DNorm from data/records_sample/records.pkl.
 - For all records: all_rows=1
 - For fixed number, all_rows=0 and n_rows=[number of rows you want]
 
This will output DNorm input data to data/DNorm_input.txt
 
Put file in DNorm folder and run DNorm

In [ ]:
df_record = pd.read_pickle(os.path.join(inputPath,'records.pkl'))

# This cell creates input in the form of
# 10192393|t|A common human skin tumour is caused by activating mutations in beta-catenin.
# 10192393|a|WNT signalling orchestrates a number of developmental programs. In response to this stimulus, cytoplasmic beta-catenin (encoded by CTNNB1) is stabilized, enabling downs$
# for the more advanced function ./ApplyDNorm

all_rows = 1
n_rows = 10

if all_rows==1:
    n_rows = len(df_record)

rec4input = df_record.loc[:n_rows,:]

outF = open(os.path.join(outputPath, 'dnorm_input.txt'), "w")
# build list of strings
rec4input = rec4input.reset_index()
for i in range(len(rec4input)):
    Id = rec4input.loc[i,:].Id
    title = rec4input.loc[i,:].title.replace('|','')
    summary = rec4input.loc[i,:].summary.replace('|','')
    # remove | characters from title and summary
    outF.write(Id+'|t|'+title)
    outF.write("\n")
    outF.write(Id+'|a|'+summary)
    outF.write("\n")

# 2. 
# Turn DNorm output into pandas df 
 - After running DNorm, put output text file in data/DNorm 
 - Save output text file as pandas dataframe


In [ ]:
#this cell takes the output from the more advanced function ./applyDNorm

output_file_path = os.path.join(outputPath, 'dnorm_output.txt')

with open(output_file_path,'r') as f:
    data = f.read().splitlines()
    
table = [d.split('\t') for d in data if len(d.split('\t'))>1]
headers = ['Id', 'start', 'end', 'disease_tag', 'tag_type', 'ONTid']
df = pd.DataFrame(table, columns=headers)

s = df.ONTid.str.split(':')
df_aux = pd.DataFrame.from_items(zip(s.index, s.values)).T
df_aux.columns = ['ont', 'unique_id']
df['ont'] = df_aux['ont']
df['unique_id'] = df_aux['unique_id']
df = df.drop('ONTid', axis = 1)

df = pd.merge(df_record[['Id', 'PDAT']], df, on='Id', how='outer')

df.to_pickle(os.path.join(pklPath,'geo_dnorm.pkl'))